In [ ]:
from db import *
import re

In [2]:
def remove_punct(lk):
    lk1=[]
    for i in lk.lower().split():
        l = (i.encode("ascii", "ignore")).decode("utf-8")
        if "\\" not in l:
            lk1.append(l)
    lk1 = " ".join(lk1)
    punctuations = '''!()-[]{};:'"\<>.,=/?@$%^&*_~|'''
    lk2 = ""
    for char in lk1:
        if char not in punctuations:
            lk2 = lk2 + char
        else:
            lk2 = lk2 + " "

    return re.sub(" +", " ", lk2)

In [3]:
def soft_skils_checker(descrption_data):
    soft_skills=[]
    l=[]
    generic_skills = skillsets_coll.find({"skill_type": "generic"})
    for i in generic_skills:
        skill=" "+remove_punct(i["name"])+" "
        if descrption_data.count(skill)>=1:
            kls=remove_punct(i["name"])
            if kls not in l:
                l.append(kls)
                soft_skills.append([kls,1])
    return soft_skills


In [4]:
def techncial_skill_checker(descrption_data):
    hard_skills=[]
    l=[]
    specific_skills = skillsets_coll.find({"skill_type": "specific"})
    for i in specific_skills:
        skill=" "+remove_punct(str(i["name"]))+" "
        if descrption_data.count(skill)>=1:
            kls = remove_punct(i["name"])
            if kls not in l:
                l.append(kls)
                hard_skills.append([kls, i["discernablity"]])
    return hard_skills

In [5]:

def student_skill_matching():
    resumes=student_resumes.find({})
    c=1
    for resume in resumes:
        
        list1=[]
        resume1=resume["preprocessed"]
        hard_skills=techncial_skill_checker(resume1)
        soft_skills=soft_skils_checker(resume1)
        student_resumes.update_one({"_id":resume["_id"]},{"$set":{"soft_skills":soft_skills,
                                                         "technical_skills":hard_skills}})
        #list1.append([c,resume['_id'],resume["preprocessed"])
        c+=1

In [6]:
def job_skill_matching():
    jobs=aracomo_jobs.find()
    for job in jobs:
        resume1=job["job_descrption"]
        hard_skills=techncial_skill_checker(resume1)
        soft_skills=soft_skils_checker(resume1)
        aracomo_jobs.update_one({"_id":job["_id"]},{"$set":{"soft_skills":soft_skills,
                                                         "technical_skills":hard_skills}})

In [11]:
def job_matching_score():
    jobs=aracomo_jobs.find()
    
    for job in jobs:
        list1=[]
        job_title = job["job_title"]
        job_title = job_title.split()
        job_title = "_".join(job_title)
        resumes = student_resumes.find()
        c=1
        for resume in resumes:
            technical_skills=[]
            for i in resume["technical_skills"]:
                for j in job["technical_skills"]:
                    if i[0]==j[0]:
                        technical_skills.append([i[0],i[1]])
            #print technical_skills

            soft_skills = []
            for i in resume["soft_skills"]:
                for j in job["soft_skills"]:
                    if i[0] == j[0]:
                        soft_skills.append([i[0],i[1]])
            job_score=0
            for j in job["technical_skills"]:
                job_score=job_score+float(j[1])
            for j in job["soft_skills"]:
                job_score=job_score+float(j[1])
            #print(job_score,job["job_title"])
            #print soft_skills
            score=0
            if len(technical_skills)==0:
                score=0
            else:
                for i in technical_skills:
                    score=score+float(i[1])
                for i in soft_skills:
                    score=score+float(i[1])
            percentage=float((score*100)/job_score)
            #print(round(percentage,2))
            student_resumes.update_one({"_id": resume["_id"]}, {"$set": {str(job_title)+"soft_skills": soft_skills,
                                                               str(job_title)+"technical_skills": technical_skills,
                                                                     str(job_title)+"score":score}})
        
            list1.append([c,resume["_id"],resume["preprocessed"],resume["soft_skills"],resume["technical_skills"],soft_skills,technical_skills,score,round(percentage,2)])
            c+=1
    return list1

In [8]:
student_skill_matching()
job_skill_matching()

In [13]:
list1=job_matching_score()
#print(list1)

In [14]:
import pandas as pd
cols=["s.no","studnet_id","student_resume","extarcted_soft_skills","extarcted_technical_skills","matched_soft_skills","matched_technical_skills","score","percentage"]

df=pd.DataFrame(list1,columns=cols)
print(df)
#cols=["s.no","studnet_id","student_resume","extarcted_soft_skills","extarcted_technical_skills","matched_soft_skills","matched_technical_skills","score","percentage"]
df.to_csv("student_job_macthing.csv")

    s.no                studnet_id  \
0      1  5bfe6c1bc625f624f2d363ef   
1      2  5bfe6c1cc625f624f2d363f0   
2      3  5bfe6c1cc625f624f2d363f1   
3      4  5bfe6c1cc625f624f2d363f2   
4      5  5bfe6c1cc625f624f2d363f3   
5      6  5bfe6c1cc625f624f2d363f4   
6      7  5bfe6c1cc625f624f2d363f5   
7      8  5bfe6c1cc625f624f2d363f6   
8      9  5bfe6c1cc625f624f2d363f7   
9     10  5bfe6c1cc625f624f2d363f8   
10    11  5bfe6c1cc625f624f2d363f9   
11    12  5bfe6c1cc625f624f2d363fa   
12    13  5bfe6c1cc625f624f2d363fb   
13    14  5bfe6c1cc625f624f2d363fc   
14    15  5bfe6c1cc625f624f2d363fd   
15    16  5bfe6c1cc625f624f2d363fe   
16    17  5bfe6c1cc625f624f2d363ff   
17    18  5bfe6c1cc625f624f2d36400   
18    19  5bfe6c1cc625f624f2d36401   
19    20  5bfe6c1cc625f624f2d36402   
20    21  5bfe6c1cc625f624f2d36403   
21    22  5bfe6c1cc625f624f2d36404   
22    23  5bfe6c1cc625f624f2d36405   
23    24  5bfe6c1cc625f624f2d36406   
24    25  5bfe6c1cc625f624f2d36407   
25    26  5b

In [16]:
from pandas import ExcelWriter
writer = pd.ExcelWriter('job_matching.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

ImportError: No module named openpyxl.workbook